# Use Random forest to drop the unimportant channels
* This notebook will use random forest to classify the data and sum the features importance obtains from random forest so that we know which channels are the most important and which channels are the least important
* The goal is to be able to using random forest as feature selection to select only the useful channels for classfication.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler
from mne.decoding import SPoC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as tts
import mne
from sklearn.model_selection import StratifiedKFold
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator,
                          cross_val_multiscore, LinearModel, get_coef)
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE 
import os
from mne.decoding import UnsupervisedSpatialFilter
import os.path as op
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from scipy.stats import skew, kurtosis
import mne 

from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import SVC  # noqa
from sklearn.model_selection import ShuffleSplit  # noqa

from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA
from scipy.stats import skew, kurtosis
from sklearn.metrics import confusion_matrix
from mne.viz import tight_layout

import time
import pandas as pd
from sklearn.ensemble import RandomForestRegressor


In [4]:
%run Batch_ArtifactFilter_Epoch.ipynb

In [5]:
epoch=remove_artifacts_and_create_epochs('D:\\USB Drive\\NewEEG-200s\\SA20140206\\20140206_1151.set')

hello
Reading D:\USB Drive\NewEEG-200s\SA20140206\20140206_1151.fdt
Reading 0 ... 831247  =      0.000 ...  1623.529 secs...
5977 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
time difference 1.80859375
[509034      0      4]
[509960      0    130]

 **1 bad trials dropped**
Setting up band-stop filter
Filter length of 3379 samples (6.600 sec) selected
Setting up band-pass filter from 5 - 1e+02 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 25.0 Hz
Filter length of 845 samples (1.650 sec) selected
5977 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   27.8s
[Parallel(n_jobs=10)]: Done 129 out of 129 | elapsed:   42.3s finished


5584 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
Fitting ICA to data using 128 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 128
Fitting ICA took 173.6s.
    Searching for artifacts...
Artifact indices found:
    
Ready.
Transforming to ICA space (128 components)
Zeroing out 0 ICA components
1979 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1979 events and 500 original time points ...
1011 bad epochs dropped


In [6]:
def removebadchannel(epoch):
    epoch.drop_channels(['Lm','Rm','VEOG'])
    epoch.drop_channels(['STI 014'])

In [7]:
 removebadchannel(epoch)

In [8]:
newdata=epoch.get_data()
epoch.get_data().shape

(968, 125, 500)

In [1]:
def filter_event_id(epoch):
    newepoch=epoch
    epochs=[[]]
    j=0
    for i in range(len(newepoch.events[:,-1])):
        if i % 3 ==0:
            epochs.append([])
            j=j+1
            epochs[j].append(epoch.events[i,-1])
        else:
            epochs[j].append(epoch.events[i,-1])
    for events in epochs:
        if  4 in events or 1 in events or 13 in events or 16 in events:
            for i in range(len(events)):
                    events[i]=4
        else:
            for i in range(len(events)):
                    events[i]=5
    epochs.pop(0)
    flattened_list = [y for x in epochs for y in x]
    for i in range(len(newepoch.events)):
        newepoch.events[i]=flattened_list[i]
    return newepoch

In [2]:
epoch=filter_event_id(epoch)
print(epoch.events[:,-1])

NameError: name 'epoch' is not defined

In [15]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

(968, 62500)


In [16]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))


Accuracy: 0.5082644628099173


In [17]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape

In [18]:
channel1.shape

(125,)

In [20]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

LL1     0.032933
RE3     0.028642
LD5     0.025103
LD2     0.025054
LL5     0.023427
LD7     0.020484
R3      0.018113
Z11     0.017973
R1      0.017915
LC7     0.017148
R7      0.015788
LC4     0.015585
RD4     0.014985
LB4     0.014439
LE3     0.014407
RC7     0.013730
LB3     0.013700
Z10     0.013603
RR3     0.013038
LA3     0.012893
LC1     0.012414
LL3     0.011947
L10     0.011773
Z9      0.011743
L2      0.011707
RR9     0.011685
RD5     0.011434
Z13     0.011213
RA2     0.011125
L4      0.011036
          ...   
LL9     0.003272
LD6     0.003154
L11     0.003135
Z6      0.003105
LD3     0.003034
RE1     0.002926
L1      0.002916
RR13    0.002909
Z14     0.002832
RR2     0.002805
LB5     0.002627
LA5     0.002574
LA4     0.002524
Z7      0.002104
RA1     0.002099
Z12     0.002095
LC3     0.001547
RR10    0.001507
LL2     0.001502
RA3     0.001402
RC2     0.001113
R4      0.001080
RA5     0.000557
RC5     0.000533
Z5      0.000527
R13     0.000518
RB1     0.000445
L8      0.0004

In [22]:
epoch.drop_channels(['LL13','RD6'])

ValueError: Channel(s) LL13, RD6 not found, nothing dropped.

In [23]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

(968, 61500)


In [28]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

Accuracy: 0.5371900826446281


In [29]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape

(123,)

In [30]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

LD5     0.026533
LL1     0.024749
RE3     0.023394
LD7     0.021925
R1      0.021638
RD4     0.021474
LL5     0.021254
Z11     0.020744
R3      0.017808
RR3     0.017703
LD2     0.017579
R7      0.016910
LB3     0.016764
LL3     0.016204
LC4     0.015759
LC7     0.015734
Z9      0.014279
RR9     0.014254
RB4     0.013621
Z10     0.013603
LA2     0.013534
LE3     0.013489
LL11    0.013057
L10     0.012995
LC6     0.012979
LL12    0.012829
RR1     0.012670
LB4     0.012545
RC1     0.012510
L13     0.012365
          ...   
RC2     0.003499
L11     0.003382
L1      0.003251
LL8     0.003156
L6      0.003147
RB1     0.003141
L3      0.002877
RD3     0.002869
L12     0.002833
RC5     0.002543
R14     0.002507
LB2     0.002389
LL7     0.002037
RR10    0.001640
LC3     0.001637
RD7     0.001570
RA3     0.001099
Z6      0.001093
R13     0.001020
Z5      0.000560
RA1     0.000559
R4      0.000558
RR2     0.000547
L8      0.000521
R12     0.000508
Z4      0.000484
LD6     0.000000
RR13    0.0000

In [32]:
epoch.drop_channels(['RR13','LD6','LA5','LL2'])

ValueError: Channel(s) RR13, LD6, LA5, LL2 not found, nothing dropped.

In [34]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

(968, 59500)


In [35]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

Accuracy: 0.5041322314049587


In [37]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape

(119,)

In [40]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

LL1     0.034656
LD7     0.026271
LD2     0.026103
RE3     0.024941
Z11     0.021891
LD5     0.021881
R3      0.019679
LL11    0.018766
RD4     0.018439
R1      0.018407
LL5     0.018171
LB3     0.017298
R7      0.016395
RC4     0.015232
Z10     0.015150
LB4     0.014864
LC4     0.014712
RB4     0.014516
LL3     0.013466
L2      0.013333
RR2     0.012367
RC1     0.012356
LA2     0.012313
L13     0.012287
RR3     0.012109
Z13     0.011812
LA3     0.011785
LC7     0.011610
Z9      0.011458
LC6     0.011448
          ...   
Z5      0.003825
LB2     0.003734
Z6      0.003627
LL9     0.003272
Z12     0.003194
L12     0.003157
L11     0.003073
RR10    0.003064
LA4     0.003021
RD7     0.002820
L7      0.002674
RA1     0.002617
LD4     0.002554
RA3     0.002397
RB1     0.002120
Z7      0.002120
L6      0.002056
L8      0.002014
LL7     0.001643
Z2      0.001115
L1      0.001072
RC5     0.000865
R4      0.000553
Z14     0.000546
LB5     0.000521
LC3     0.000474
R13     0.000000
RA5     0.0000

In [41]:
epoch.drop_channels(['R13','RA5','LD3','RE1'])

<Epochs  |   968 events (all good), 0 - 1.94922 sec, baseline [None, 0], ~424.9 MB, data loaded,
 'b-f1-pic': 0
 'b-f1-snd': 0
 'b-f1-spk': 425
 'b-f1-wrd': 0
 'b-f2-pic': 0
 'b-f2-snd': 0
 'b-f2-spk': 543
 'b-f2-wrd': 0
 'b-tg-pic': 0
 'b-tg-snd': 0
 'b-tg-spk': 0
 'b-tg-wrd': 0
 'f-f1-pic': 0
 'f-f1-snd': 0
 'f-f1-spk': 0
 'f-f1-wrd': 0
 'f-f2-pic': 0
 'f-f2-snd': 0
 'f-f2-spk': 0
 'f-f2-wrd': 0
 'f-tg-pic': 0
 'f-tg-snd': 0
 'f-tg-spk': 0
 'f-tg-wrd': 0>

In [42]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

(968, 57500)


In [44]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

Accuracy: 0.5247933884297521


In [45]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape

(115,)

In [46]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

LL1     0.025655
LD7     0.025542
RE3     0.023577
R1      0.021592
LD5     0.020276
Z11     0.020220
LL3     0.020022
LD2     0.019878
R7      0.018433
R3      0.018205
LL5     0.017861
LC7     0.017702
RR1     0.016296
LC4     0.016202
LC6     0.014478
L2      0.014291
LA2     0.014214
RC7     0.014171
Z9      0.014063
LE1     0.013845
LB3     0.013700
LB4     0.013611
RD4     0.013503
RD2     0.013168
LL11    0.012604
RR9     0.012551
RC4     0.012397
Z10     0.011934
LB1     0.011877
R8      0.011810
          ...   
R6      0.004120
Z12     0.004082
LB2     0.003503
RA1     0.003491
RR10    0.003402
Z2      0.003202
LL8     0.003156
RD3     0.002869
L7      0.002742
LL7     0.002687
RC2     0.002666
RA3     0.002558
RC5     0.002557
R14     0.002507
RB1     0.002503
L11     0.002163
RR7     0.002133
Z6      0.002113
LD4     0.002106
RD7     0.002091
L1      0.002043
L6      0.001572
L8      0.001536
Z7      0.001364
R12     0.001118
R4      0.001081
LC3     0.000935
LB6     0.0000

In [47]:
epoch.drop_channels(['LB6','Z5','Z4'])

<Epochs  |   968 events (all good), 0 - 1.94922 sec, baseline [None, 0], ~413.9 MB, data loaded,
 'b-f1-pic': 0
 'b-f1-snd': 0
 'b-f1-spk': 425
 'b-f1-wrd': 0
 'b-f2-pic': 0
 'b-f2-snd': 0
 'b-f2-spk': 543
 'b-f2-wrd': 0
 'b-tg-pic': 0
 'b-tg-snd': 0
 'b-tg-spk': 0
 'b-tg-wrd': 0
 'f-f1-pic': 0
 'f-f1-snd': 0
 'f-f1-spk': 0
 'f-f1-wrd': 0
 'f-f2-pic': 0
 'f-f2-snd': 0
 'f-f2-spk': 0
 'f-f2-wrd': 0
 'f-tg-pic': 0
 'f-tg-snd': 0
 'f-tg-spk': 0
 'f-tg-wrd': 0>

In [48]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

(968, 56000)


In [49]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

Accuracy: 0.5578512396694215


In [50]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape

(112,)

In [51]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

LL1     0.028295
LD7     0.023866
RE3     0.023007
LD2     0.021128
Z11     0.021092
LL5     0.020795
R1      0.020558
LD5     0.020276
R3      0.019730
LB3     0.017217
Z10     0.016811
LC7     0.016638
LL3     0.016432
R7      0.015925
L2      0.015431
LC4     0.015233
L10     0.014511
RD5     0.014491
LB4     0.014418
LC6     0.014228
RR3     0.014092
RC7     0.013787
L4      0.013266
RD4     0.013028
LL4     0.012937
Z13     0.012834
Z9      0.012426
LC1     0.011680
RR6     0.011232
LA2     0.010994
          ...   
RE4     0.004954
LB1     0.004840
LD4     0.004675
LL6     0.004587
RR2     0.004569
L3      0.004472
LA4     0.004429
L8      0.004149
R14     0.004058
R11     0.003986
RB1     0.003983
RA1     0.003864
LB5     0.003787
RD7     0.003373
LL9     0.003272
LL8     0.003156
R12     0.002939
R9      0.002927
RD3     0.002869
RR7     0.002099
L1      0.001645
L6      0.001572
RA3     0.001298
RC2     0.001113
LC3     0.001086
LL7     0.001060
R4      0.000523
RR10    0.0004

In [52]:
epoch.drop_channels(['Z2','Z6'])

<Epochs  |   968 events (all good), 0 - 1.94922 sec, baseline [None, 0], ~406.5 MB, data loaded,
 'b-f1-pic': 0
 'b-f1-snd': 0
 'b-f1-spk': 425
 'b-f1-wrd': 0
 'b-f2-pic': 0
 'b-f2-snd': 0
 'b-f2-spk': 543
 'b-f2-wrd': 0
 'b-tg-pic': 0
 'b-tg-snd': 0
 'b-tg-spk': 0
 'b-tg-wrd': 0
 'f-f1-pic': 0
 'f-f1-snd': 0
 'f-f1-spk': 0
 'f-f1-wrd': 0
 'f-f2-pic': 0
 'f-f2-snd': 0
 'f-f2-spk': 0
 'f-f2-wrd': 0
 'f-tg-pic': 0
 'f-tg-snd': 0
 'f-tg-spk': 0
 'f-tg-wrd': 0>

In [53]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

(968, 55000)


In [54]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

Accuracy: 0.5743801652892562


In [55]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape

(110,)

In [56]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

LL1     0.035766
RE3     0.024009
Z11     0.023854
LD5     0.023399
LD2     0.023200
RD4     0.020960
LD7     0.020951
R1      0.019018
LL5     0.019008
LB3     0.017934
R7      0.017307
L2      0.016931
R3      0.016755
LA3     0.016733
LC6     0.016597
LC4     0.016518
Z10     0.015691
L3      0.015068
LB4     0.014033
Z9      0.013934
RR3     0.013577
LL3     0.013466
R8      0.012838
LE1     0.012689
L13     0.012530
L10     0.012300
RR9     0.012038
RE4     0.011865
RC1     0.011717
RR4     0.011660
          ...   
Z12     0.005032
RB5     0.005002
LL4     0.004964
RB2     0.004951
Z14     0.004794
LB2     0.004754
R9      0.004485
RB3     0.004423
LD4     0.004260
R2      0.004036
L12     0.003677
RB1     0.003660
LL8     0.003635
RC5     0.003519
R11     0.003496
RD3     0.003358
RD7     0.002960
RA1     0.002680
LB5     0.002522
L1      0.001964
Z7      0.001536
LA4     0.001214
L8      0.001096
LL7     0.000972
RR10    0.000841
L7      0.000558
R4      0.000554
L11     0.0005

In [57]:
epoch.drop_channels(['RA3'])

<Epochs  |   968 events (all good), 0 - 1.94922 sec, baseline [None, 0], ~402.8 MB, data loaded,
 'b-f1-pic': 0
 'b-f1-snd': 0
 'b-f1-spk': 425
 'b-f1-wrd': 0
 'b-f2-pic': 0
 'b-f2-snd': 0
 'b-f2-spk': 543
 'b-f2-wrd': 0
 'b-tg-pic': 0
 'b-tg-snd': 0
 'b-tg-spk': 0
 'b-tg-wrd': 0
 'f-f1-pic': 0
 'f-f1-snd': 0
 'f-f1-spk': 0
 'f-f1-wrd': 0
 'f-f2-pic': 0
 'f-f2-snd': 0
 'f-f2-spk': 0
 'f-f2-wrd': 0
 'f-tg-pic': 0
 'f-tg-snd': 0
 'f-tg-spk': 0
 'f-tg-wrd': 0>

In [58]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

(968, 54500)


In [59]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

Accuracy: 0.5289256198347108


In [60]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape

(109,)

In [61]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

LL1     0.030251
RE3     0.023813
LD5     0.023490
LD2     0.023283
LD7     0.021357
LL5     0.020957
R1      0.019747
LA3     0.019701
LC4     0.019063
R7      0.018932
R3      0.018835
LC7     0.018637
RD4     0.018443
RR9     0.018180
Z11     0.016899
LL3     0.015989
L2      0.015365
LC6     0.014379
RR3     0.014373
RD2     0.014289
RR11    0.013763
LB3     0.013700
Z13     0.013266
LB4     0.012965
L10     0.012962
LC1     0.012656
Z9      0.012507
LE3     0.012416
Z10     0.012374
LL11    0.012000
          ...   
RC5     0.005402
Z14     0.005134
RB2     0.005024
LC2     0.004969
RC3     0.004905
R6      0.004668
R14     0.004661
LL4     0.004490
Z12     0.004454
RA1     0.003976
R2      0.003468
L1      0.003254
R4      0.003038
R9      0.002927
RD3     0.002869
R12     0.002811
L6      0.002698
LL7     0.002680
L12     0.002556
LA4     0.002465
LB2     0.002389
RC2     0.002170
L11     0.002163
RB1     0.001943
LC3     0.001591
RD7     0.001570
L8      0.001546
RD5     0.0010

In [62]:
epoch.drop_channels(['LB5','RR10'])

<Epochs  |   968 events (all good), 0 - 1.94922 sec, baseline [None, 0], ~395.4 MB, data loaded,
 'b-f1-pic': 0
 'b-f1-snd': 0
 'b-f1-spk': 425
 'b-f1-wrd': 0
 'b-f2-pic': 0
 'b-f2-snd': 0
 'b-f2-spk': 543
 'b-f2-wrd': 0
 'b-tg-pic': 0
 'b-tg-snd': 0
 'b-tg-spk': 0
 'b-tg-wrd': 0
 'f-f1-pic': 0
 'f-f1-snd': 0
 'f-f1-spk': 0
 'f-f1-wrd': 0
 'f-f2-pic': 0
 'f-f2-snd': 0
 'f-f2-spk': 0
 'f-f2-wrd': 0
 'f-tg-pic': 0
 'f-tg-snd': 0
 'f-tg-spk': 0
 'f-tg-wrd': 0>

In [63]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

(968, 53500)


In [64]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

Accuracy: 0.5413223140495868


In [68]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
print(channel1.shape)
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

(107,)


LL1     0.030052
RE3     0.026692
LD5     0.025006
LD7     0.023110
Z11     0.020778
Z10     0.019804
LL5     0.019705
LD2     0.019369
R1      0.019030
R3      0.018835
RD4     0.017899
R7      0.016228
RC7     0.015999
RE4     0.015541
LB3     0.014940
LC4     0.014712
LL11    0.014702
RR9     0.014470
RR3     0.014054
Z9      0.013956
LC7     0.013902
LL3     0.013717
L2      0.013187
LE3     0.013131
LE4     0.013063
RD2     0.012872
LB4     0.012837
LC1     0.012323
L4      0.011725
Z13     0.011713
          ...   
LC5     0.005429
LL9     0.005219
RD3     0.005175
LB2     0.005131
LB1     0.004840
LD4     0.004640
L14     0.004533
R9      0.004474
RB2     0.004469
RR11    0.004383
LL8     0.004275
Z12     0.004035
RR7     0.004015
R11     0.003986
RC2     0.003584
RR2     0.003553
R12     0.003535
L1      0.003423
L12     0.003368
RD7     0.003133
LA4     0.002951
RB1     0.002851
RC5     0.001982
L8      0.001866
Z7      0.001543
R4      0.001427
LL7     0.001078
L6      0.0008

In [69]:
epoch.drop_channels(['L11','LC3'])

<Epochs  |   968 events (all good), 0 - 1.94922 sec, baseline [None, 0], ~388.0 MB, data loaded,
 'b-f1-pic': 0
 'b-f1-snd': 0
 'b-f1-spk': 425
 'b-f1-wrd': 0
 'b-f2-pic': 0
 'b-f2-snd': 0
 'b-f2-spk': 543
 'b-f2-wrd': 0
 'b-tg-pic': 0
 'b-tg-snd': 0
 'b-tg-spk': 0
 'b-tg-wrd': 0
 'f-f1-pic': 0
 'f-f1-snd': 0
 'f-f1-spk': 0
 'f-f1-wrd': 0
 'f-f2-pic': 0
 'f-f2-snd': 0
 'f-f2-spk': 0
 'f-f2-wrd': 0
 'f-tg-pic': 0
 'f-tg-snd': 0
 'f-tg-spk': 0
 'f-tg-wrd': 0>

In [70]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

(968, 52500)


In [71]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
print(channel1.shape)
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

Accuracy: 0.5330578512396694
(105,)


LL1       0.034170
RE3       0.024371
LD5       0.023064
LD7       0.022550
LD2       0.021558
R1        0.020562
LL5       0.019146
L2        0.017923
RD4       0.017899
Z11       0.017418
R3        0.016755
R7        0.016603
LA3       0.015984
LC4       0.015601
Z10       0.015171
L10       0.015034
Z3        0.014829
LC7       0.014234
RR9       0.014114
LB3       0.013700
R8        0.013584
LE3       0.013443
LL11      0.013170
Z9        0.013040
RR3       0.013038
L13       0.012851
LB4       0.012095
LE1       0.012082
LL10      0.011421
RR11      0.011349
            ...   
L12       0.005802
RR5       0.005709
R14       0.005635
R6        0.005579
L14       0.005515
L6        0.005442
RC2       0.005441
Z8        0.005384
L7        0.005227
LL7       0.005123
RD7       0.005068
LL4       0.005043
Nasium    0.005027
R9        0.004786
RB5       0.004478
RB2       0.004469
RC3       0.004454
LB2       0.004079
R12       0.003369
LL8       0.003156
Z12       0.003058
L8        0.

In [73]:
epoch.drop_channels(['RC5','R2'])

<Epochs  |   968 events (all good), 0 - 1.94922 sec, baseline [None, 0], ~380.6 MB, data loaded,
 'b-f1-pic': 0
 'b-f1-snd': 0
 'b-f1-spk': 425
 'b-f1-wrd': 0
 'b-f2-pic': 0
 'b-f2-snd': 0
 'b-f2-spk': 543
 'b-f2-wrd': 0
 'b-tg-pic': 0
 'b-tg-snd': 0
 'b-tg-spk': 0
 'b-tg-wrd': 0
 'f-f1-pic': 0
 'f-f1-snd': 0
 'f-f1-spk': 0
 'f-f1-wrd': 0
 'f-f2-pic': 0
 'f-f2-snd': 0
 'f-f2-spk': 0
 'f-f2-wrd': 0
 'f-tg-pic': 0
 'f-tg-snd': 0
 'f-tg-spk': 0
 'f-tg-wrd': 0>

In [74]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
print(channel1.shape)
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

(968, 51500)
Accuracy: 0.5206611570247934
(103,)


LL1     0.029185
LD5     0.028656
RE3     0.024468
LD7     0.024177
R3      0.023829
LB3     0.019614
LD2     0.019055
Z11     0.018895
LL5     0.018615
LC4     0.018453
RD4     0.017899
LL11    0.017795
Z10     0.017625
R1      0.017023
R7      0.016862
RR9     0.015451
RR4     0.014218
L10     0.013662
RC7     0.013253
RR3     0.013038
LC6     0.013015
L2      0.012781
LB4     0.012667
Z3      0.012557
RB3     0.012452
RC4     0.012195
LC1     0.012174
LA2     0.012046
RR11    0.011762
Z13     0.011625
          ...   
LL4     0.006374
L5      0.006362
LD4     0.006344
L7      0.006110
Z8      0.006050
RB5     0.005510
LL8     0.005457
LB1     0.005324
L9      0.005189
R9      0.005065
L14     0.005051
RB2     0.005023
RC3     0.005012
L8      0.004776
LL6     0.004655
RR2     0.004647
LB2     0.004137
R6      0.004120
LL12    0.004049
RA1     0.004045
LL7     0.003153
Z7      0.002636
L1      0.002634
Z12     0.002570
LA4     0.002465
RD7     0.001624
RC2     0.001121
Z14     0.0005

In [75]:
epoch.drop_channels(['RB1','R4','Z14','RC2','RD7','LA4'])

<Epochs  |   968 events (all good), 0 - 1.94922 sec, baseline [None, 0], ~358.4 MB, data loaded,
 'b-f1-pic': 0
 'b-f1-snd': 0
 'b-f1-spk': 425
 'b-f1-wrd': 0
 'b-f2-pic': 0
 'b-f2-snd': 0
 'b-f2-spk': 543
 'b-f2-wrd': 0
 'b-tg-pic': 0
 'b-tg-snd': 0
 'b-tg-spk': 0
 'b-tg-wrd': 0
 'f-f1-pic': 0
 'f-f1-snd': 0
 'f-f1-spk': 0
 'f-f1-wrd': 0
 'f-f2-pic': 0
 'f-f2-snd': 0
 'f-f2-spk': 0
 'f-f2-wrd': 0
 'f-tg-pic': 0
 'f-tg-snd': 0
 'f-tg-spk': 0
 'f-tg-wrd': 0>

In [76]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
print(channel1.shape)
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

(968, 48500)
Accuracy: 0.5495867768595041
(97,)


LL1       0.034216
LD2       0.027626
LD5       0.026676
R1        0.025719
RE3       0.023590
LD7       0.021040
LL5       0.020555
L2        0.020140
RD4       0.019422
Z11       0.018900
R3        0.017746
Z10       0.015602
LL11      0.015452
LB4       0.015090
LC4       0.015085
Z9        0.015042
LB3       0.014595
LA3       0.014472
LL3       0.014326
RR3       0.014089
RC4       0.013701
LC6       0.013463
R7        0.013182
LE3       0.013166
RR9       0.013029
Z1        0.012560
LD1       0.012553
RR11      0.012429
LL10      0.012138
RB3       0.011931
            ...   
RC6       0.007071
L7        0.006838
R14       0.006755
LD4       0.006292
LB1       0.006290
LL4       0.005931
R11       0.005899
LA1       0.005817
RC3       0.005560
LC2       0.005559
Z3        0.005501
RA4       0.005242
LB2       0.005218
LL6       0.005141
L6        0.005096
RA1       0.005022
RB2       0.005018
Z8        0.004933
RD3       0.004306
LL8       0.004223
Nasium    0.004074
R9        0.

In [77]:
epoch.drop_channels(['L8','LL7','L1','Z12','L12','Z7','Nasium'])

<Epochs  |   968 events (all good), 0 - 1.94922 sec, baseline [None, 0], ~332.6 MB, data loaded,
 'b-f1-pic': 0
 'b-f1-snd': 0
 'b-f1-spk': 425
 'b-f1-wrd': 0
 'b-f2-pic': 0
 'b-f2-snd': 0
 'b-f2-spk': 543
 'b-f2-wrd': 0
 'b-tg-pic': 0
 'b-tg-snd': 0
 'b-tg-spk': 0
 'b-tg-wrd': 0
 'f-f1-pic': 0
 'f-f1-snd': 0
 'f-f1-spk': 0
 'f-f1-wrd': 0
 'f-f2-pic': 0
 'f-f2-snd': 0
 'f-f2-spk': 0
 'f-f2-wrd': 0
 'f-tg-pic': 0
 'f-tg-snd': 0
 'f-tg-spk': 0
 'f-tg-wrd': 0>

In [78]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),500):
    channel.append(np.sum(importances[i:i+500]))
    channel1=np.array(channel)
    channel1.shape
print(channel1.shape)
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

(968, 45000)
Accuracy: 0.5247933884297521
(90,)


LL1     0.032374
RE3     0.029172
LD2     0.027270
LD5     0.025737
LD7     0.024792
Z11     0.023263
R7      0.021718
R1      0.020389
LL3     0.019370
RD4     0.019282
Z10     0.017922
LL5     0.017343
R3      0.017312
RD2     0.017273
L2      0.016588
LE3     0.016013
LC7     0.015975
LB4     0.015540
LC4     0.014712
RR1     0.014625
L10     0.014615
LA3     0.014315
LL11    0.014081
LB3     0.014076
RC7     0.012672
RR9     0.012322
L14     0.012312
LC6     0.012195
Z3      0.012108
RR11    0.012073
          ...   
LL12    0.007817
RB4     0.007713
LE4     0.007699
RD1     0.007314
R14     0.007159
LC2     0.007153
L9      0.007102
LB2     0.006928
L5      0.006922
RR8     0.006856
RC3     0.006832
RR5     0.006770
LC5     0.006535
R9      0.006358
LL4     0.006324
Z8      0.006007
RB3     0.005912
L7      0.005892
RA4     0.005363
R12     0.005009
LL6     0.005008
LB1     0.004840
RA1     0.004408
R11     0.003981
RB5     0.003959
R10     0.003869
LL8     0.003700
RD5     0.0036